<a href="https://colab.research.google.com/github/Ris-code/BasicML/blob/main/Yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'dcsass-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F205791%2F451400%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240503%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240503T110640Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D669002c274de0c080567f24f9734867621c56cd8f517befa96049db19043032d31da6eae8f8ec9617197cefb96a095b11ac2e050b2204f16818cd26f776fb4becf4e9371fbb6ece59c4d36ff726f266d2b2d7ccb7bbb3989d6c5692b457d678222174efd8bfd817848927e777367ea0ebae743217153a1a6e71a6cba5e7952307c4751d38bf3eabd1004323f74c45087b865b149cc05548a013bb9eac85ac7871879f0fa84d87053a55f5c70110d15f1905107be366fb66f8e8ee211b99152692f190fe576567bcd107a80960f52783d7d95bf9c5a4051122f30d2e3c7d756922ed3bd389217a34a1d4de8eaab86e144037f9e24d60af83329360c106a4d810b'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import cv2
import torch
from torchvision import transforms
from model import Darknet
from utils.utils import non_max_suppression, load_classes

# Load YOLOv3 model
model = Darknet("config/yolov3.cfg")
model.load_weights("weights/yolov3.weights")
model.eval()

# Load classes
classes = load_classes("data/coco.names")

# Transformation for input images
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Function to detect objects in a frame
def detect_objects(frame):
    img = transform(frame).unsqueeze(0)
    with torch.no_grad():
        detections = model(img)
        detections = non_max_suppression(detections, conf_thres=0.5, nms_thres=0.4)
    return detections[0]

# Process video
video_path = "/kaggle/input/dcsass-dataset/RoadAccidents/RoadAccidents001_x264.mp4"
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    detections = detect_objects(frame)

    if detections is not None:
        for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections:
            label = classes[int(cls_pred)]
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'model'

In [ ]:
pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 44.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.1/755.1 kB 28.4 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.5.0
    Uninstalling Pillow-9.5.0:
      Successfully uninstalled Pillow-9.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
import cv2
from PIL import Image

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Load video
video_path = '/kaggle/input/dcsass-dataset/RoadAccidents/RoadAccidents001_x264.mp4'
video_capture = cv2.VideoCapture(video_path)
print(1)

while video_capture.isOpened():
    print(1)
    ret, frame = video_capture.read()
    if not ret:
        break

    # Convert frame to PIL image
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Perform object detection
    results = model(pil_image)

    # Draw bounding boxes and labels on the frame
    annotated_image = results.render()[0]

    # Convert back to OpenCV format
    annotated_frame = cv2.cvtColor(np.array(annotated_image), cv2.COLOR_RGB2BGR)

    # Display annotated frame
    cv2.imshow('YOLOv5 Object Detection', annotated_frame)

    # Press q to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close windows
video_capture.release()
# cv2.destroyAllWindows()


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-5-3 Python-3.10.13 torch-2.1.2+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


1


In [ ]:
import torch
import cv2
from PIL import Image

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Load video
video_path = 'your_video.mp4'
video_capture = cv2.VideoCapture(video_path)

while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
        break

    # Convert frame to PIL image
    pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Perform object detection
    results = model(pil_image)

    # Draw bounding boxes and labels on the frame
    annotated_image = results.render()[0]

    # Convert back to OpenCV format
    annotated_frame = cv2.cvtColor(np.array(annotated_image), cv2.COLOR_RGB2BGR)

    # Display annotated frame
    cv2.imshow('YOLOv5 Object Detection', annotated_frame)

    # Press q to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture
video_capture.release()


In [ ]:
import torch
import cv2
from PIL import Image
import numpy as np

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Load video
video_path = '/kaggle/input/dcsass-dataset/DCSASS Dataset/RoadAccidents/RoadAccidents001_x264.mp4/RoadAccidents001_x264_10.mp4'
video_capture = cv2.VideoCapture(video_path)

# Check if video capture object is opened successfully
if not video_capture.isOpened():
    print("Error: Unable to open video file.")
    exit()

# Get video properties
fps = int(video_capture.get(cv2.CAP_PROP_FPS))
width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
output_video_path = 'annotated_video.mp4'

# Define video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Read the first frame
ret, frame = video_capture.read()

# Check if frame is read successfully
if not ret:
    print("Error: Unable to read frame from video.")
    exit()

# Convert frame to PIL image
pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# Perform object detection
results = model(pil_image)

# Draw bounding boxes and labels on the frame
annotated_image = results.render()[0]

# Convert annotated frame back to OpenCV format
annotated_frame = cv2.cvtColor(np.array(annotated_image), cv2.COLOR_RGB2BGR)

# Write annotated frame to output video
output_video.write(annotated_frame)

# Release video capture and writer
video_capture.release()
output_video.release()

# Print message
print(f"Annotated video saved at: {output_video_path}")


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-5-3 Python-3.10.13 torch-2.1.2+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Annotated video saved at: annotated_video.mp4


In [ ]:
import cv2

def play_video(video_path):
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        cv2.imshow('Video', frame)

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    cap.release()
#     cv2.destroyAllWindows()

video_path = '/kaggle/working/annotated_video.mp4'  # Replace 'path_to_your_video.mp4' with the path to your video file
play_video(video_path)

error: OpenCV(4.9.0) /io/opencv/modules/highgui/src/window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
